# Lab 5: Ensemble Machine Learning – Wine Dataset
**Author:** Eric Meyer

**Date:** 04/09/2025  

**Objective:**

## Section 1: Import and Inspect the Data

In [1]:
# Imports
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    BaggingClassifier,
    VotingClassifier,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

In [8]:
# Load the wine quality dataset
df = pd.read_csv("winequality-red.csv", sep=";")

# Display basic structure

df.info()
print
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


## Section 2: Data Exploration and Preparation

### 2.1 Handle Missing Values and Clean Data

In [9]:
# Convert quality score to labels: low, medium, high
def quality_to_label(q):
    if q <= 4:
        return "low"
    elif q <= 6:
        return "medium"
    else:
        return "high"

df["quality_label"] = df["quality"].apply(quality_to_label)

In [10]:
# Convert quality score to numeric class: 0 = low, 1 = medium, 2 = high
def quality_to_number(q):
    if q <= 4:
        return 0
    elif q <= 6:
        return 1
    else:
        return 2

df["quality_numeric"] = df["quality"].apply(quality_to_number)

# Display updated DataFrame
print("\nUpdated Dataset Sample:")
print(df[["quality", "quality_label", "quality_numeric"]].head())


Updated Dataset Sample:
   quality quality_label  quality_numeric
0        5        medium                1
1        5        medium                1
2        5        medium                1
3        6        medium                1
4        5        medium                1


## Section 3: Feature Selection and Justification

In [11]:
# Input features: all except target and label columns
X = df.drop(columns=["quality", "quality_label", "quality_numeric"])
y = df["quality_numeric"]

# Explanation:
# We removed the original quality score and the text label to focus on a numeric target (0, 1, 2).
# Features are physicochemical properties such as acidity, alcohol, pH, etc.

## Section 4: Split the Data into Train and Test

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Section 5: Evaluate Model Performance - Random Forest (100) and Gradient Boosting (100)

In [13]:
# Define helper function to train and evaluate models
def evaluate_model(name, model, X_train, y_train, X_test, y_test, results):
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    train_f1 = f1_score(y_train, y_train_pred, average="weighted")
    test_f1 = f1_score(y_test, y_test_pred, average="weighted")

    print(f"\n{name} Results")
    print("Confusion Matrix (Test):")
    print(confusion_matrix(y_test, y_test_pred))
    print(f"Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")
    print(f"Train F1 Score: {train_f1:.4f}, Test F1 Score: {test_f1:.4f}")

    results.append(
        {
            "Model": name,
            "Train Accuracy": train_acc,
            "Test Accuracy": test_acc,
            "Train F1": train_f1,
            "Test F1": test_f1,
            "Accuracy Gap": train_acc - test_acc,
            "F1 Score Gap": train_f1 - test_f1,
        }
    )

# Evaluate two ensemble models
results = []

In [14]:
# Random Forest (100)
evaluate_model(
    "Random Forest (100)",
    RandomForestClassifier(n_estimators=100, random_state=42),
    X_train,
    y_train,
    X_test,
    y_test,
    results,
)


Random Forest (100) Results
Confusion Matrix (Test):
[[  0  13   0]
 [  0 256   8]
 [  0  15  28]]
Train Accuracy: 1.0000, Test Accuracy: 0.8875
Train F1 Score: 1.0000, Test F1 Score: 0.8661


In [15]:
# Gradient Boosting (100)
evaluate_model(
    "Gradient Boosting (100)",
    GradientBoostingClassifier(
        n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42
    ),
    X_train,
    y_train,
    X_test,
    y_test,
    results,
)


Gradient Boosting (100) Results
Confusion Matrix (Test):
[[  0  13   0]
 [  3 247  14]
 [  0  16  27]]
Train Accuracy: 0.9601, Test Accuracy: 0.8562
Train F1 Score: 0.9584, Test F1 Score: 0.8411


## Section 6: Compare Results

In [16]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Test Accuracy", ascending=False)

print("\nComparison of Model Performance:")
print(results_df)


Comparison of Model Performance:
                     Model  Train Accuracy  Test Accuracy  Train F1   Test F1  \
0      Random Forest (100)        1.000000        0.88750   1.00000  0.866056   
1  Gradient Boosting (100)        0.960125        0.85625   0.95841  0.841106   

   Accuracy Gap  F1 Score Gap  
0      0.112500      0.133944  
1      0.103875      0.117304  


## Section 7: Conclusion and Insights

After evaluating ensemble learning models to predict the quality of red wine, I found that **Random Forest (100 trees)** outperformed **Gradient Boosting (100 trees)** in terms of **Test Accuracy (0.8875 vs. 0.8563)** and **F1 Score (0.8661 vs. 0.8411)**. However, both models showed signs of overfitting, with noticeable performance gaps between training and testing sets.

| Model                   | Train Accuracy | Test Accuracy | Train F1 | Test F1 | Accuracy Gap | F1 Score Gap |
|------------------------|----------------|---------------|----------|---------|---------------|--------------|
| Random Forest (100)    | 1.0000         | 0.8875        | 1.0000   | 0.8661  | 0.1125        | 0.1339       |
| Gradient Boosting (100)| 0.9601         | 0.8563        | 0.9584   | 0.8411  | 0.1039        | 0.1173       |

### Model Comparison and Interpretation

- **Random Forest** perfectly fits the training data, suggesting high variance. Despite this, it generalizes surprisingly well on the test set, likely due to the averaging effect of many decorrelated trees.
- **Gradient Boosting** has slightly lower training accuracy but still achieves a solid generalization score. Its iterative improvement process may give it more robustness on unseen data, especially with tuning.

---

### Final Takeaway

While Random Forest is the top performer based on raw metrics, **Gradient Boosting shows more promise for tuning and scalability**. It’s slightly less prone to overfitting and can be fine-tuned for even better generalization.

As an analyst, my role is not just to select the best model but to **tell the story of why it performs well, what trade-offs are at play, and how this informs next steps for decision-making.** This project reinforced how ensemble learning brings both power and nuance to real-world predictions like wine quality.